<a href="https://colab.research.google.com/github/NataKrj/AI-project-2024/blob/main/Step_1_Step_2_Step_3_sanction_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Step -Sanction list check

In [13]:


import pandas as pd

# URLs for the CSV files
url1 = 'https://www.treasury.gov/ofac/downloads/sdn.csv'
url2 = 'https://www.treasury.gov/ofac/downloads/consolidated/cons_alt.csv'

# Read data from the first CSV file
df1 = pd.read_csv(url1, on_bad_lines='skip')
sanction_list_url1 = df1.iloc[:, 1].dropna().unique()

# Read data from the second CSV file
df2 = pd.read_csv(url2, on_bad_lines='skip')
sanction_list_url2 = df2.iloc[:, 3].dropna().unique()

# Combine the names from both CSV files
sanction_list = list(set(sanction_list_url1) | set(sanction_list_url2))

# Create a DataFrame from the combined names
sanction_list_df = pd.DataFrame({'Sanctioned Names': sanction_list})

# Save the DataFrame to a CSV file
sanction_list_df.to_csv('sanction_list.csv', index=False)

print("Sanctioned names saved to 'sanction_list.csv'")

Sanctioned names saved to 'sanction_list.csv'


In [14]:
!pip install fuzzywuzzy
import pandas as pd
from fuzzywuzzy import fuzz

# Paths to the files
uploaded_file_path = 'BELGIUM_companies_short.xlsx'
sanction_list_file_path = 'sanction_list.csv'

# Load the companies file
companies_df = pd.read_excel(uploaded_file_path)

# Load the sanction list
sanction_list_df = pd.read_csv(sanction_list_file_path)

# Normalize the sanction list for case-insensitive matching
sanctioned_names = sanction_list_df['Sanctioned Names'].str.lower().tolist()

# Function for approximate matching
def approximate_match(name, sanctioned_names, threshold=85):
    """ Check if a name approximately matches any sanctioned name.
    : name: Name to match
    :sanctioned_names: List of sanctioned names
    :threshold: Minimum similarity score for a match
    : 30 if a match is found, 0 otherwise
    """
    name = name.lower()
    for sanctioned_name in sanctioned_names:
        similarity = fuzz.ratio(name, sanctioned_name)
        if similarity >= threshold:
            return 30  # Match found
    return 0  # No match

# Evaluate if company names approximately match any name in the sanction list
companies_df['Score_Step_1'] = companies_df['Name'].apply(
    lambda name: approximate_match(name, sanctioned_names)
)

# Save the updated  new file
output_file_path = 'Step_1_evaluated_companies.xlsx'
companies_df.to_excel(output_file_path, index=False)

print(f"Evaluation complete with approximate matching. Results saved to {output_file_path}")



Evaluation complete with approximate matching. Results saved to Step_1_evaluated_companies.xlsx


# II STEP- Company status/ active check

In [15]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from time import sleep
from multiprocessing import Pool

def process_companies(company_chunk):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 20)

    base_url = "https://kbopub.economie.fgov.be/kbopub/zoeknaamfonetischform.html?lang=en"
    result_chunk = []
    successful_count = 0

    company_types = [
        "VZW", "BVBA", "BV", "NV", "CV", "CVBA", "SPRL", "SCRL", "ASBL",
        "Comm.V", "SComm", "VOF", "SNC", "GIE", "AIE", "SE", "Partnership"
    ]

    def clean_company_name(company_name):
        return re.sub(r'\b(?:' + '|'.join(company_types) + r')\b', '', company_name, flags=re.IGNORECASE).strip()

    sleep_time = 10
    for company_name in company_chunk:
        try:
            clean_name = clean_company_name(company_name)
            driver.get(base_url)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            sleep(sleep_time)
            search_box = wait.until(EC.presence_of_element_located((By.ID, "searchWord")))
            search_box.clear()
            search_box.send_keys(clean_name)

            checkbox = driver.find_element(By.ID, "filterEnkelActieve")
            if checkbox.is_selected():
                checkbox.click()

            search_button = wait.until(EC.element_to_be_clickable((By.NAME, "actionNPRP")))
            search_button.click()
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

            try:
                page_text = driver.find_element(By.TAG_NAME, "body").text
                if "no result found for this search term.".lower() in page_text.lower():
                    print(f"No result for {company_name}")
                    result_chunk.append({
                        'OriginalCompanyName': company_name,
                        'CleanedCompanyName': clean_name,
                        'Status': "No result found for this search term",
                        'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    continue
            except NoSuchElementException:
                pass

            rows = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#onderneminglistfonetisch tbody tr')))
            status = "not found in KBO data table"
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            for row in rows:
                name_cell = row.find_element(By.CLASS_NAME, 'benaming').text.strip()
                if name_cell.lower() == clean_name.lower():
                    status_cell = row.find_elements(By.TAG_NAME, 'td')[1].text.strip()
                    status = re.sub(r'\s+', ' ', status_cell).strip()
                    successful_count += 1
                    break

            result_chunk.append({
                'OriginalCompanyName': company_name,
                'CleanedCompanyName': clean_name,
                'Status': status,
                'Timestamp': timestamp
            })

        except (NoSuchElementException, TimeoutException, Exception) as e:
            print(f"Failed to process {company_name}")
            result_chunk.append({
                'OriginalCompanyName': company_name,
                'CleanedCompanyName': clean_name,
                'Status': "error",
                'Timestamp': "N/A"
            })

    driver.quit()
    return result_chunk, successful_count


if __name__ == '__main__':
    start_time = datetime.now()
    print(f"Start time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

    # Load the Excel file
    uploaded_file_path = 'BELGIUM_companies_short.xlsx'  # Replace with your actual file path
    company_list = pd.read_excel(uploaded_file_path)['Name']

    num_workers = 10

    # Split the list of companies into chunks for multiprocessing
    company_chunks = np.array_split(company_list, num_workers)
    with Pool(num_workers) as pool:
        results = pool.map(process_companies, company_chunks)

    # Combine all results
    all_results = [item[0] for item in results]
    successful_count = sum(item[1] for item in results)
    result_df = pd.DataFrame([item for sublist in all_results for item in sublist])

    # Define the scoring dictionary
    status_scores = {
        "ENT LP Active": 1,
        "ENT LP Stopped": 5,
        "error": 2,
        "EU Active": 1,
        "EU Stopped": 5,
        "No result found for this search term": 2,
        "not found in KBO data table": 2
    }

    # Map the 'Status' column to scores based on the dictionary
    result_df['Score'] = result_df['Status'].map(status_scores).fillna(0)

    # Save the updated DataFrame to a new CSV file
    result_df.to_csv('Step_2_company_status_report_with_scores.csv', index=False)

    end_time = datetime.now()
    print(f"End time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total time taken: {end_time - start_time}")
    print(f"Total successfully found statuses: {successful_count}")



Start time: 2024-12-03 15:40:30


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


No result for Zwick Roell Belux CV
Failed to process Albatros Zweefvliegclub
No result for Brugs Motoren Bedrijf nv
No result for Van Laer-Mazet/Chris
Failed to process Zzlite
End time: 2024-12-03 15:41:39
Total time taken: 0:01:08.513871
Total successfully found statuses: 12


# III STEP- web scrapping

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, Comment
from concurrent.futures import ThreadPoolExecutor, as_completed
import csv
import re
from googleapiclient.discovery import build
from requests.exceptions import RequestException, SSLError

# Set up your API keys
google_api_key = 'xxx'  # Replace with your Google API key
google_cse_id = 'xxx'  # Replace with your Custom Search Engine ID

# Load the CSV file to get company names
df = pd.read_csv('Offshore Leaks-entities.csv', low_memory=False, encoding='utf-8')
company_names = df['name'][0:20].tolist()  # Processing the first 20 companies

# Keywords and associated scores
keywords_score_30 = [
    "sanctions", "criminal", "crime", "corruption", "shell company", "criminal case", "arrested"
]
keywords_score_5 = [
    "court", "accusation", "penalty", "investigation", "insolvency", "violation", "debt", "blackmail"
]
keywords_score_0 = ["stock", "stock price"]

score_no_words = 1

def google_search(search_term, api_key, cse_id, start_index=1):
    service = build("customsearch", "v1", developerKey=api_key)
    try:
        res = service.cse().list(q=search_term, cx=cse_id, start=start_index).execute()
        return res.get('items', [])
    except Exception as e:
        print(f"Failed to search for {search_term} with error: {e}")
        return []

def extract_text_from_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5'
    }
    try:
        response = requests.get(url, headers=headers, verify=False, timeout=10)
        if response.status_code == 200:
            if 'text/html' in response.headers.get('Content-Type', ''):
                soup = BeautifulSoup(response.text, 'html.parser')
                for script in soup(["script", "style", "header", "footer", "form", "nav"]):
                    script.extract()
                for comment in soup.findAll(text=lambda text: isinstance(text, Comment)):
                    comment.extract()
                text = ' '.join(soup.stripped_strings)
                return text
            else:
                return "Non-text content skipped"
        else:
            return ""
    except (RequestException, SSLError) as e:
        return f"Request failed for {url}: {e}"

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace with single space
    text = text.strip()
    return text

def calculate_score(text):
    """
    Determine the score for a given text based on keyword matching.
    """
    text_lower = text.lower()
    if any(keyword in text_lower for keyword in keywords_score_30):
        return 30
    elif any(keyword in text_lower for keyword in keywords_score_5):
        return 5
    elif any(keyword in text_lower for keyword in keywords_score_0):
        return 0
    elif text.strip() == "":
        return score_no_words
    else:
        return score_no_words

def process_company(company_name):
    results = google_search(company_name, google_api_key, google_cse_id)
    company_data = []
    for result in results:
        url = result['link']
        extracted_text = extract_text_from_url(url)
        if extracted_text != "Non-text content skipped":
            extracted_text = clean_text(extracted_text)
            score = calculate_score(extracted_text)
            company_data.append({
                'company': company_name,
                'url': url,
                'extracted_text': extracted_text,
                'score': score
            })
        else:
            company_data.append({
                'company': company_name,
                'url': url,
                'extracted_text': "Skipped due to non-text content",
                'score': score_no_words
            })
    return company_data

# Use ThreadPoolExecutor to process companies in parallel
data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(process_company, name): name for name in company_names}
    for future in as_completed(futures):
        data.extend(future.result())

# Convert list of dicts to DataFrame
df_results = pd.DataFrame(data)

# Save the DataFrame to a CSV file with proper encoding and escaping
output_file_path = 'Step_3_company_analysis_with_scores.csv'  # Replace with desired file path
df_results.to_csv(output_file_path, index=False, escapechar='\\', encoding='utf-8', quoting=csv.QUOTE_ALL)

print(f"Data saved to {output_file_path}.")


